#### 필요한 라이브러리 불러오기

In [ ]:
from skimage import filters
import cv2
from scipy import ndimage
from skimage.filters import threshold_otsu
import numpy as np
import pandas as pd
from keras.models import load_model
import tensorflow_addons as tfa

#### 추론 함수

In [ ]:
def inference(MODEL_LOAD_PATH,DATE_PRODUCT,thickness,length):
    path = "/root/work/hyundairb/products/"+ DATE_PRODUCT + "/"
    # 전체 파일 경로 
    file_path = [path+i for i in os.listdir(path)]
    # 확장자가 tif인 파일들만 가져오기 
    file_path = [i for i in file_path if i[-3:]=='tif']
    # (1)번 카메라로 찍은 이미지들
    first_list = [i for i in file_path if i.split('/')[-1].split('-')[5]=="(1)"]
    
    # (1)번 카메라로 찍은 이미지들을 컷 번호 순서대로 다시 정렬
    new_first_list = first_list.copy()
    for i in first_list:
        num = int(i.split('.')[0].split('/')[-1].split('-')[6])
        new_first_list[num-1] = i
    
    # (2)번 카메라로 찍은 이미지들
    second_list = [i for i in file_path if i.split('/')[-1].split('-')[5]=="(2)"]
    
    # (2)번 카메라로 찍은 이미지들을 컷 번호 순서대로 다시 정렬
    new_second_list = second_list.copy()
    for i in second_list:
        num = int(i.split('.')[0].split('/')[-1].split('-')[6])
        new_second_list[num-1] = i
    
    # 전체 파일 경로를 합치기 ((1)번 카메라 컷 번호 순서대로 정리한 것 + (2)번 카메라 컷 번호 순서대로 정리한 것)
    file_path = new_first_list + new_second_list
    num_path = len(file_path)
    print(num_path)
    
    # 인자로 받은 전체 파이프 길이를 mm 단위로 변경하고 이미지 갯수로 나누어서 한 픽셀 당 길이 구하기 
    pixel_len = (length*10) / num_path
    
    # 시간 단축을 위해 추론에서는 2장의 이미지로만 경계 추출을 하고 그 경계 좌표를 모든 이미지들에 일괄 적용
    # (1)번 카메라에서 하나 뽑기 
    # 1번은 컷 자국이 있고 2번은 숫자가 있어서 그 두 장을 제외하고 나머지 이미지들 중 첫번째 뽑기 
    first_criteria = [i for i in file_path if (i.split('/')[-1].split('-')[5]=='(1)')&
                      (i.split('/')[-1].split('.')[0].split('-')[6]!='1')&
                      (i.split('/')[-1].split('.')[0].split('-')[6]!='2')][0]
    first_criteria_img = cv2.imread(first_criteria, 0)
    # 용접부분 경계선 추출
    first_Dog = filters.difference_of_gaussians(first_criteria_img, 12,12*16)
    size = 90
    first_minimum_trib = ndimage.minimum_filter(first_Dog, size)
    first_orig_sub_min = first_Dog - first_minimum_trib
    first_thresh = threshold_otsu(first_orig_sub_min)
    first_binary = first_orig_sub_min > first_thresh
    first_bool_to_pix = np.where(first_binary==True,255,0)
    first_bool_to_pix = np.array(first_bool_to_pix, dtype = "uint8")
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(60,60))
    first_opening = cv2.morphologyEx(first_bool_to_pix, cv2.MORPH_OPEN, rect_kernel)
    first_upper = np.max(np.where(first_opening==255)[0])+50
    first_under = np.min(np.where(first_opening==255)[0])-50
    
    # (2)번 카메라에서 하나 뽑기 
    second_criteria = [i for i in file_path if (i.split('/')[-1].split('-')[5]=='(2)')][0]
    second_criteria_img = cv2.imread(second_criteria, 0)
    second_Dog = filters.difference_of_gaussians(second_criteria_img, 12,12*16)
    second_minimum_trib = ndimage.minimum_filter(second_Dog, size)
    second_orig_sub_min = second_Dog - second_minimum_trib
    second_thresh = threshold_otsu(second_orig_sub_min)
    second_binary = second_orig_sub_min > second_thresh
    second_bool_to_pix = np.where(second_binary==True,255,0)
    second_bool_to_pix = np.array(second_bool_to_pix, dtype = "uint8")
    second_opening = cv2.morphologyEx(second_bool_to_pix, cv2.MORPH_OPEN, rect_kernel)
    second_upper = np.max(np.where(second_opening==255)[0])+50
    second_under = np.min(np.where(second_opening==255)[0])-50
    
    # 경계선 좌표는 첫 번째 기준과 두 번째 기준의 각각 평균 좌표로 구함 
    upper = int(np.mean([first_upper,second_upper]))
    under = int(np.mean([first_under,second_under]))
    
    processed = []
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    for f in file_path:
        img = cv2.imread(f,0)
        # clahe 처리한 이미지들 저장
        equalized = clahe.apply(img)
        save_path = "/root/work/hdrb/hdrb_node/node_python/data/WE_DRT_CUT/"
        
        # 만약 해당 날짜 폴더가 존재하면
        if os.path.exists(save_path + DATE_PRODUCT.split('/')[0]):
            # 그 날짜 폴더 안에 해당 product명이 존재하면
            if os.path.exists(save_path + DATE_PRODUCT):
                # 거기에 평활화된 이미지를 저장 
                cv2.imwrite(save_path+DATE_PRODUCT+"/"+f.split('/')[-1].split('.')[0]+'.png', equalized)
                # 저장이 되었음을 확인하는 출력 
                print(save_path+DATE_PRODUCT+"/"+f.split('/')[-1].split('.')[0]+'.png')
            # 만약 그 날짜 폴더에 해당 product명이 존재하지 않는다면 
            else:
                # 해당 product명으로 된 폴더를 하나 생성하고
                os.makedirs(save_path + DATE_PRODUCT)
                # 거기에 평활화된 이미지를 저장
                cv2.imwrite(save_path+DATE_PRODUCT+"/"+f.split('/')[-1].split('.')[0]+'.png', equalized)
                # 저장이 되었음을 확인하는 출력
                print(save_path+DATE_PRODUCT+"/"+f.split('/')[-1].split('.')[0]+'.png')
        # 만약 해당 날짜 폴더가 존재하지 않는다면 
        else:
            # 해당 날짜 폴더 생성하고
            os.makedirs(save_path + DATE_PRODUCT.split('/')[0])
            # 그 아래에 해당 product명으로 된 폴더 생성하고 
            os.makedirs(save_path + DATE_PRODUCT)
            # 거기에 평활화된 이미지를 저장
            cv2.imwrite(save_path+DATE_PRODUCT+"/"+f.split('/')[-1].split('.')[0]+'.png', equalized)
            # 저장이 되었음을 확인하는 출력
            print(save_path+DATE_PRODUCT+"/"+f.split('/')[-1].split('.')[0]+'.png')
            
            
        # 이미지의 색상을 10구간으로 나누고 각 구간에 각각 10개의 색상으로 채움
        cut_img = img[under:upper,:]
        section = pd.cut(np.concatenate(cut_img).tolist(), 10, labels = [0,28,56,84,112,140,168,196,224,252])
        processed_img = np.array(section).reshape(np.array(cut_img).shape)
        # 모델에 넣기 위해 정규화
        normalized_img = processed_img.astype('float32') / 255.
        # 이미지를 (400,400)으로 resize 
        resized_img = cv2.resize(normalized_img,(400,400),cv2.INTER_AREA)
        
        processed.append(resized_img)
        
    # 벡엔드에서 넘겨받은 경로의 모델을 로드 
    model = load_model(MODEL_LOAD_PATH, custom_objects={'f1_score':tfa.metrics.FBetaScore(num_classes=2,average="micro",threshold=0.5)})
    
    # 추론 시작
    yhat = model.predict(np.array(processed))
    
    # 추론 결과 라벨 달기 
    result = []
    for i in yhat:
        if(i.argmax()==1):
            l = "R"
        else:
            l = "A"
        result.append(l)
        
    num_list = []
    for i in range(num_path):
        num_list.append(num_path*-1+i)
        
    # 벡엔드에서 출력값을 파싱하기 편하도록 함수 만들기
    # ***.**** 으로 소숫점 앞은 세자리까지, 소숫점 뒤는 네자리까지 표기 하도록 
    def make_output(number):
        if(number>=1000):
            number = '999.0000'
        else:
            if(number//10==0):
                number = '00'+"{:.4f}".format(number)
            elif(number//100==0):
                number = '0'+"{:.4f}".format(number)
            else:
                number = "{:.4f}".format(number)
        return number
    
    # 추론 결과 정보 출력
    for i in range(len(file_path)):
        print("img:", file_path[i].split('/')[-1].split('.')[0], 
              "CLASS:", result[i], "Prob:", 
              make_output(yhat[i][yhat[i].argmax()]),
              "length:", str(int(np.floor(i*pixel_len)))+'-'+str(int(np.floor((i+1)*pixel_len))),
              "index", i,
              "minus index" , num_list[i]
               )